In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
import numpy as np
from scipy.spatial import distance

from tqdm.auto import tqdm
import pickle
import gc
import json
import h5py
import pandas as pd

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# == recnn ==
import sys
sys.path.append("../../")
import recnn


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../../data/parsed/omdb.json'))
tqdm.pandas()

In [4]:
env = recnn.env.FrameEnv('../../data/embeddings/ml20_pca128.pkl',
                         '../../data/ml-20m/ratings.csv', 10, 1)

In [5]:
ddpg = recnn.nn.models.Actor(1290, 128, 256).to(cuda)
td3 = recnn.nn.models.Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('../../models/ddpg_policy.pt'))
td3.load_state_dict(torch.load('../../models/td3_policy.pt'))

<All keys matched successfully>

In [6]:
test_batch = next(iter(env.test_dataloader))
state = test_batch[0].to(cuda)

In [50]:
def rank(gen_action, metric):
    scores = []
    for i in env.movie_embeddings_key_dict.keys():
        if i == 0 or i == '0':
            continue
        scores.append([i, metric(env.movie_embeddings_key_dict[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])]['omdb'][key]  for key in ['Title',
                                'Genre', 'Language', 'Released', 'imdbRating']])
        # scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'Title', 'Genre', 'Language', 'Released', 'imdbRating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

In [51]:
'0' in env.movie_embeddings_key_dict.keys()

False

# DDPG

In [52]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [53]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,Title,Genre,Language,Released,imdbRating
0,1226,38.454380,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
1,86418,38.693405,City Streets,"Crime, Drama, Film-Noir, Romance",English,18 Apr 1931,7.2
2,2238,38.705502,Seven Beauties,"Comedy, Drama, War","Italian, German, Neapolitan, Spanish",21 Jan 1976,7.9
3,26243,38.713840,Evil Roy Slade,"Comedy, Western",English,18 Feb 1972,7.3
4,127587,38.717018,El siete machos,"Comedy, Western",Spanish,02 May 1951,7.2
5,4039,38.723839,Annie,"Comedy, Drama, Family, Musical",English,18 Jun 1982,6.6
6,3249,38.807106,The Hand That Rocks the Cradle,"Drama, Thriller",English,10 Jan 1992,6.6
7,130089,38.912762,Crazy Beautiful You,"Drama, Romance","Filipino, Tagalog",25 Feb 2015,7.3
8,1313,39.000637,Mad Dog Time,"Comedy, Crime, Drama",English,08 Nov 1996,5.5
9,114958,39.003056,Moomins on the Riviera,"Animation, Comedy, Family",Finnish,04 Dec 2015,6.0


In [54]:
rank(ddpg_action, distance.cosine)

,id,score,Title,Genre,Language,Released,imdbRating
0,3249,0.692846,The Hand That Rocks the Cradle,"Drama, Thriller",English,10 Jan 1992,6.6
1,4579,0.695717,Cookie,"Comedy, Crime",English,23 Aug 1989,5.3
2,1226,0.698382,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
3,124693,0.701924,Hell's Highway,"Action, Crime, Drama",English,23 Sep 1932,6.8
4,4039,0.703932,Annie,"Comedy, Drama, Family, Musical",English,18 Jun 1982,6.6
5,114958,0.704148,Moomins on the Riviera,"Animation, Comedy, Family",Finnish,04 Dec 2015,6.0
6,127587,0.708250,El siete machos,"Comedy, Western",Spanish,02 May 1951,7.2
7,7179,0.708879,Wuthering Heights,"Drama, History, Romance",English,16 Oct 1992,6.8
8,2238,0.709273,Seven Beauties,"Comedy, Drama, War","Italian, German, Neapolitan, Spanish",21 Jan 1976,7.9
9,112395,0.710581,Belle Starr,Western,English,12 Sep 1941,5.8


In [55]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,Title,Genre,Language,Released,imdbRating
0,3249,0.693113,The Hand That Rocks the Cradle,"Drama, Thriller",English,10 Jan 1992,6.6
1,4579,0.698211,Cookie,"Comedy, Crime",English,23 Aug 1989,5.3
2,1226,0.700020,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
3,114958,0.704181,Moomins on the Riviera,"Animation, Comedy, Family",Finnish,04 Dec 2015,6.0
4,124693,0.704493,Hell's Highway,"Action, Crime, Drama",English,23 Sep 1932,6.8
5,4039,0.706158,Annie,"Comedy, Drama, Family, Musical",English,18 Jun 1982,6.6
6,7179,0.710350,Wuthering Heights,"Drama, History, Romance",English,16 Oct 1992,6.8
7,127587,0.710475,El siete machos,"Comedy, Western",Spanish,02 May 1951,7.2
8,2238,0.711836,Seven Beauties,"Comedy, Drama, War","Italian, German, Neapolitan, Spanish",21 Jan 1976,7.9
9,112395,0.712745,Belle Starr,Western,English,12 Sep 1941,5.8


In [56]:
rank(ddpg_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,63329,78.813468,Christmas Story,"Drama, Family, Fantasy",Finnish,16 Nov 2007,7.0
1,93894,80.093110,The Unknown Woman,"Documentary, War",Finnish,04 Feb 2011,6.5
2,113943,80.161511,House of Women,"Crime, Drama",English,06 Sep 1962,6.2
3,86233,80.583126,New Kids Turbo,"Action, Comedy",Dutch,09 Dec 2010,6.4
4,26459,80.719636,The Scarlet Pimpernel,"Action, Drama, Romance",English,09 Nov 1982,7.7
5,117561,81.923760,Once Upon a Time in Queens,"Comedy, Drama",English,08 Jun 2013,6.3
6,8003,82.172795,Bedlam,"Drama, Horror, Thriller",English,10 May 1946,6.9
7,1276,82.229112,Cool Hand Luke,"Crime, Drama",English,01 Nov 1967,8.1
8,90170,82.264571,Monsieur Vincent,"Biography, Drama, History",French,20 Dec 1948,7.1
9,88692,82.511611,Princess Kaiulani,Drama,"English, Hawaiian",14 May 2010,6.2


In [57]:
rank(ddpg_action, distance.minkowski)

,id,score,Title,Genre,Language,Released,imdbRating
0,1226,38.454380,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
1,86418,38.693405,City Streets,"Crime, Drama, Film-Noir, Romance",English,18 Apr 1931,7.2
2,2238,38.705502,Seven Beauties,"Comedy, Drama, War","Italian, German, Neapolitan, Spanish",21 Jan 1976,7.9
3,26243,38.713840,Evil Roy Slade,"Comedy, Western",English,18 Feb 1972,7.3
4,127587,38.717018,El siete machos,"Comedy, Western",Spanish,02 May 1951,7.2
5,4039,38.723839,Annie,"Comedy, Drama, Family, Musical",English,18 Jun 1982,6.6
6,3249,38.807106,The Hand That Rocks the Cradle,"Drama, Thriller",English,10 Jan 1992,6.6
7,130089,38.912762,Crazy Beautiful You,"Drama, Romance","Filipino, Tagalog",25 Feb 2015,7.3
8,1313,39.000637,Mad Dog Time,"Comedy, Crime, Drama",English,08 Nov 1996,5.5
9,114958,39.003056,Moomins on the Riviera,"Animation, Comedy, Family",Finnish,04 Dec 2015,6.0


In [58]:
rank(ddpg_action, distance.chebyshev)

,id,score,Title,Genre,Language,Released,imdbRating
0,8507,7.855177,Freaks,"Drama, Horror","English, German, French",20 Feb 1932,7.9
1,102716,8.164085,Fast & Furious 6,"Action, Adventure, Thriller","English, Spanish, Russian, Japanese, Cantonese...",24 May 2013,7.1
2,7235,8.187895,Ichi the Killer,"Action, Crime, Drama, Horror, Thriller","Japanese, Cantonese, English",25 Jul 2002,7.0
3,8983,8.192516,House of Flying Daggers,"Action, Adventure, Drama, Romance",Mandarin,14 Jan 2005,7.5
4,87977,8.193316,Murder in Coweta County,"Crime, Drama",English,15 Feb 1983,7.6
5,50358,8.202363,Invisible Waves,"Drama, Thriller","Thai, English, Japanese, Korean",02 Mar 2006,6.7
6,67534,8.215271,Big Stan,"Action, Comedy",English,06 Jun 2008,6.2
7,5138,8.229914,State Property,"Drama, Crime, Action",English,18 Jan 2002,4.5
8,39768,8.248912,Life Is a Miracle,"Comedy, Romance, Music, Drama, War","Serbian, English, German, Hungarian",14 May 2004,7.6
9,1219,8.258968,Psycho,"Horror, Mystery, Thriller",English,08 Sep 1960,8.5


In [59]:
rank(ddpg_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,117561,0.750924,Once Upon a Time in Queens,"Comedy, Drama",English,08 Jun 2013,6.3
1,105526,0.755349,Year Zero: The Silent Death of Cambodia,"Documentary, History, War","English, Khmer",30 Oct 1979,8.4
2,93894,0.756143,The Unknown Woman,"Documentary, War",Finnish,04 Feb 2011,6.5
3,26459,0.757820,The Scarlet Pimpernel,"Action, Drama, Romance",English,09 Nov 1982,7.7
4,63329,0.759775,Christmas Story,"Drama, Family, Fantasy",Finnish,16 Nov 2007,7.0
5,88692,0.760790,Princess Kaiulani,Drama,"English, Hawaiian",14 May 2010,6.2
6,113943,0.766573,House of Women,"Crime, Drama",English,06 Sep 1962,6.2
7,124693,0.768107,Hell's Highway,"Action, Crime, Drama",English,23 Sep 1932,6.8
8,89217,0.769043,Waiting for Forever,"Comedy, Drama, Romance",English,04 Feb 2011,6.0
9,99568,0.769559,Great Escape: The Untold Story,"Documentary, Drama, War",English,N/A,7.4


In [60]:
rank(ddpg_action, distance.cityblock)

,id,score,Title,Genre,Language,Released,imdbRating
0,113943,333.076355,House of Women,"Crime, Drama",English,06 Sep 1962,6.2
1,89217,342.065308,Waiting for Forever,"Comedy, Drama, Romance",English,04 Feb 2011,6.0
2,106234,343.114502,Great Expectations,"Drama, Romance",English,08 Nov 2013,6.4
3,127587,343.668701,El siete machos,"Comedy, Western",Spanish,02 May 1951,7.2
4,2238,343.712158,Seven Beauties,"Comedy, Drama, War","Italian, German, Neapolitan, Spanish",21 Jan 1976,7.9
5,59995,345.119507,Boy A,Drama,English,04 Sep 2008,7.6
6,117561,345.316345,Once Upon a Time in Queens,"Comedy, Drama",English,08 Jun 2013,6.3
7,4039,345.334473,Annie,"Comedy, Drama, Family, Musical",English,18 Jun 1982,6.6
8,4579,345.734070,Cookie,"Comedy, Crime",English,23 Aug 1989,5.3
9,41688,345.771057,Cowards Bend the Knee,"Drama, Romance",English,29 Dec 2004,7.1


# TD3

In [61]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [62]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,Title,Genre,Language,Released,imdbRating
0,99901,13.569754,The Almighty Tycoon of the Ninja,"Action, Comedy, Sci-Fi",Greek,09 Mar 2007,5.5
1,95230,13.612562,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
2,108495,13.646015,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8
3,87063,13.791531,Bis zum Ellenbogen,Comedy,German,01 Nov 2007,6.4
4,77734,13.804186,Confessions of a Gangsta,Action,English,26 Sep 2006,4.3
5,103498,13.859569,Double Solitaire,Drama,English,16 Jan 1974,N/A
6,1316,13.971952,Anna Meintjies,Drama,English,13 Nov 1996,N/A
7,112844,13.994721,Bus Stop,Comedy,English,22 Aug 1982,6.9
8,111601,14.042746,Die Fischerin,Drama,German,23 May 2014,6.3
9,80160,14.176217,The Flintstone Kids' Just Say No Special,Animation,English,15 Sep 1988,5.6


In [63]:
rank(td3_action, distance.cosine)

,id,score,Title,Genre,Language,Released,imdbRating
0,104829,0.622235,The Story of Luke,"Comedy, Drama",English,05 Apr 2013,7.2
1,89545,0.631429,X: Night of Vengeance,"Crime, Drama, Thriller",English,24 Nov 2011,5.6
2,81451,0.637052,Night Wolf,"Action, Horror, Thriller",English,01 Apr 2012,4.5
3,8530,0.646749,Dear Frankie,"Drama, Romance","English, British Sign Language",15 Apr 2005,7.8
4,97128,0.657150,A Safe Place,Drama,English,29 Oct 1971,5.3
5,94403,0.661074,The Greening of Whitney Brown,"Adventure, Family",English,18 Jan 2015,5.3
6,1585,0.665105,Love Serenade,"Comedy, Crime, Drama, Romance",English,01 Aug 1997,7.0
7,99708,0.665339,The Shunning,Drama,English,16 Apr 2011,6.4
8,6851,0.665348,Gas Food Lodging,"Drama, Romance","English, Spanish",10 Jul 1992,6.7
9,6334,0.666975,Blue Car,Drama,English,02 May 2003,6.7


In [64]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,Title,Genre,Language,Released,imdbRating
0,104829,0.619829,The Story of Luke,"Comedy, Drama",English,05 Apr 2013,7.2
1,89545,0.633051,X: Night of Vengeance,"Crime, Drama, Thriller",English,24 Nov 2011,5.6
2,81451,0.635068,Night Wolf,"Action, Horror, Thriller",English,01 Apr 2012,4.5
3,8530,0.647797,Dear Frankie,"Drama, Romance","English, British Sign Language",15 Apr 2005,7.8
4,97128,0.658922,A Safe Place,Drama,English,29 Oct 1971,5.3
5,94403,0.660738,The Greening of Whitney Brown,"Adventure, Family",English,18 Jan 2015,5.3
6,99708,0.661042,The Shunning,Drama,English,16 Apr 2011,6.4
7,6334,0.665879,Blue Car,Drama,English,02 May 2003,6.7
8,1585,0.666529,Love Serenade,"Comedy, Crime, Drama, Romance",English,01 Aug 1997,7.0
9,6851,0.667196,Gas Food Lodging,"Drama, Romance","English, Spanish",10 Jul 1992,6.7


In [65]:
rank(td3_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,4137,79.158543,Morgan Stewart's Coming Home,Comedy,English,20 Feb 1987,5.4
1,45081,80.403732,Silent Hill,Horror,English,21 Apr 2006,6.6
2,52285,80.926102,Firehouse Dog,"Action, Comedy, Family",English,04 Apr 2007,5.5
3,5831,80.991094,Soul Assassin,"Crime, Thriller, Action",English,24 Jul 2002,4.6
4,91840,81.610087,Roadie,"Comedy, Drama, Music",English,06 Jan 2012,6.0
5,2923,81.680179,Citizens Band,"Comedy, Drama",English,13 Nov 1981,6.7
6,90118,81.880745,The Sheep Has Five Legs,Comedy,"French, English",24 Sep 1954,6.8
7,102965,82.003924,Attack on the Iron Coast,"Action, Drama, War",English,05 Jul 1968,5.5
8,2095,82.024837,The Shaggy D.A.,"Comedy, Family, Fantasy",English,17 Dec 1976,5.9
9,31590,82.095086,Touchez Pas au Grisbi,"Crime, Drama",French,01 Feb 1960,7.8


In [66]:
rank(td3_action, distance.minkowski)

,id,score,Title,Genre,Language,Released,imdbRating
0,99901,13.569754,The Almighty Tycoon of the Ninja,"Action, Comedy, Sci-Fi",Greek,09 Mar 2007,5.5
1,95230,13.612562,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
2,108495,13.646015,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8
3,87063,13.791531,Bis zum Ellenbogen,Comedy,German,01 Nov 2007,6.4
4,77734,13.804186,Confessions of a Gangsta,Action,English,26 Sep 2006,4.3
5,103498,13.859569,Double Solitaire,Drama,English,16 Jan 1974,N/A
6,1316,13.971952,Anna Meintjies,Drama,English,13 Nov 1996,N/A
7,112844,13.994721,Bus Stop,Comedy,English,22 Aug 1982,6.9
8,111601,14.042746,Die Fischerin,Drama,German,23 May 2014,6.3
9,80160,14.176217,The Flintstone Kids' Just Say No Special,Animation,English,15 Sep 1988,5.6


In [67]:
rank(td3_action, distance.chebyshev)

,id,score,Title,Genre,Language,Released,imdbRating
0,129723,3.018724,Cose da pazzi,Comedy,Italian,24 Mar 2005,6.4
1,119874,3.055999,L'allenatore nel pallone 2,Comedy,Italian,11 Jan 2008,3.7
2,108495,3.101138,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8
3,99901,3.229287,The Almighty Tycoon of the Ninja,"Action, Comedy, Sci-Fi",Greek,09 Mar 2007,5.5
4,87063,3.238699,Bis zum Ellenbogen,Comedy,German,01 Nov 2007,6.4
5,77734,3.401255,Confessions of a Gangsta,Action,English,26 Sep 2006,4.3
6,101483,3.438390,Simpaticul domn R,"Action, Crime",Romanian,19 Jan 1970,5.3
7,95230,3.441968,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
8,111601,3.518709,Die Fischerin,Drama,German,23 May 2014,6.3
9,80160,3.547344,The Flintstone Kids' Just Say No Special,Animation,English,15 Sep 1988,5.6


In [68]:
rank(td3_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,4137,0.691922,Morgan Stewart's Coming Home,Comedy,English,20 Feb 1987,5.4
1,104829,0.720845,The Story of Luke,"Comedy, Drama",English,05 Apr 2013,7.2
2,85572,0.729475,The New Daughter,"Horror, Thriller",English,07 Oct 2010,5.3
3,100570,0.757217,U.S. Seals,"Action, Thriller",English,18 Jan 2000,4.9
4,97128,0.765195,A Safe Place,Drama,English,29 Oct 1971,5.3
5,52285,0.766682,Firehouse Dog,"Action, Comedy, Family",English,04 Apr 2007,5.5
6,4090,0.776299,The Brave Little Toaster,"Animation, Adventure, Family, Fantasy, Musical",English,10 Jul 1987,7.3
7,6334,0.776385,Blue Car,Drama,English,02 May 2003,6.7
8,81150,0.776922,The Trouble with Dee Dee,Comedy,English,N/A,7.0
9,5407,0.777390,La bûche,"Comedy, Drama","French, English, Russian",24 Nov 1999,6.4


In [70]:
rank(td3_action, distance.cityblock)

,id,score,Title,Genre,Language,Released,imdbRating
0,112844,121.549133,Bus Stop,Comedy,English,22 Aug 1982,6.9
1,103498,122.578918,Double Solitaire,Drama,English,16 Jan 1974,N/A
2,107836,123.937668,Annie Get Your Gun,Musical,English,19 Mar 1967,7.2
3,99901,124.068619,The Almighty Tycoon of the Ninja,"Action, Comedy, Sci-Fi",Greek,09 Mar 2007,5.5
4,113256,124.256927,The New Jean-Claude,Comedy,French,26 Jun 2002,5.4
5,1316,124.465309,Anna Meintjies,Drama,English,13 Nov 1996,N/A
6,95230,124.494522,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
7,93885,124.645432,Osmosis,"Comedy, Romance, Drama","French, English",05 May 2004,6.0
8,109030,125.319908,Métastases,Comedy,French,01 Mar 2012,6.0
9,108495,126.443100,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8


# BCQ

In [71]:
perturbator = recnn.models.bcqPerturbator(1290, 128, 256).to(cuda)
generator = recnn.models.bcqGenerator(1290, 128, 512).to(cuda)

perturbator.load_state_dict(torch.load('../../models/bcq_perturbator.pt'))
generator.load_state_dict(torch.load('../../models/bcq_generator.pt'))

<All keys matched successfully>

In [72]:
sampled_actions = generator.decode(state)
perturbed_actions= perturbator(state, sampled_actions)

In [73]:
bcq_action = perturbed_actions[np.random.randint(0,
                               sampled_actions.size(0), 1)[0]].detach().cpu().numpy()

In [74]:
rank(bcq_action, distance.euclidean)

,id,score,Title,Genre,Language,Released,imdbRating
0,3340,27.769102,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
1,3638,27.866560,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
2,2270,28.175209,A Century of Cinema,Documentary,English,N/A,5.6
3,113128,28.272356,Return to Life,Documentary,English,20 Jul 1938,6.0
4,77734,28.288343,Confessions of a Gangsta,Action,English,26 Sep 2006,4.3
5,97304,28.448215,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
6,122458,28.585938,Supernova,"Action, Adventure, Drama","English, French",05 Sep 2005,3.7
7,80160,28.630526,The Flintstone Kids' Just Say No Special,Animation,English,15 Sep 1988,5.6
8,5497,28.633619,Our Man Flint,"Action, Adventure, Comedy, Fantasy, Sci-Fi","English, Italian",16 Jan 1966,6.5
9,2683,28.641691,Austin Powers: The Spy Who Shagged Me,"Action, Adventure, Comedy, Crime","English, German",11 Jun 1999,6.6


In [75]:
rank(bcq_action, distance.cosine)

,id,score,Title,Genre,Language,Released,imdbRating
0,3638,0.614605,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
1,102156,0.620820,Zombies of the Stratosphere,"Action, Sci-Fi",English,16 Jul 1952,5.7
2,3340,0.627883,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
3,3659,0.632156,Quatermass 2,"Sci-Fi, Horror",English,01 Sep 1957,6.9
4,97304,0.642555,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
5,102090,0.642729,The Magnetic Monster,Sci-Fi,English,18 Feb 1953,5.8
6,2376,0.646069,A View to a Kill,"Action, Adventure, Thriller","English, French",24 May 1985,6.4
7,71254,0.650713,Gamer,"Action, Sci-Fi, Thriller",English,04 Sep 2009,5.8
8,8878,0.650853,Class of Nuke 'Em High Part II: Subhumanoid Me...,"Comedy, Horror, Sci-Fi",English,12 Apr 1991,4.4
9,66808,0.651965,Far Cry,"Action, Adventure, Sci-Fi",English,02 Oct 2008,3.1


In [76]:
rank(bcq_action, distance.correlation)

,id,score,Title,Genre,Language,Released,imdbRating
0,3638,0.614694,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
1,102156,0.626638,Zombies of the Stratosphere,"Action, Sci-Fi",English,16 Jul 1952,5.7
2,3340,0.628047,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
3,3659,0.634837,Quatermass 2,"Sci-Fi, Horror",English,01 Sep 1957,6.9
4,102090,0.640015,The Magnetic Monster,Sci-Fi,English,18 Feb 1953,5.8
5,97304,0.640677,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
6,2376,0.642237,A View to a Kill,"Action, Adventure, Thriller","English, French",24 May 1985,6.4
7,71254,0.646934,Gamer,"Action, Sci-Fi, Thriller",English,04 Sep 2009,5.8
8,122458,0.653172,Supernova,"Action, Adventure, Drama","English, French",05 Sep 2005,3.7
9,1255,0.653200,Bad Taste,"Comedy, Horror, Sci-Fi","English, Maori",01 Jun 1989,6.6


In [77]:
rank(bcq_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,1258,77.604380,The Shining,"Drama, Horror",English,13 Jun 1980,8.4
1,85505,78.561295,X Games 3D: The Movie,"Documentary, Sport",English,21 Aug 2009,6.4
2,5586,78.943230,The Gypsy Moths,"Action, Drama, Romance",English,13 Sep 1969,6.4
3,540,79.097669,Sliver,"Drama, Thriller",English,21 May 1993,5.0
4,4511,79.673246,High Spirits,"Comedy, Fantasy",English,18 Nov 1988,5.8
5,3174,80.188363,Man on the Moon,"Biography, Comedy, Drama",English,22 Dec 1999,7.4
6,85268,80.409850,Scooby-Doo! The Mystery Begins,"Animation, Adventure, Comedy, Family, Fantasy,...",English,13 Sep 2009,5.4
7,524,80.586946,Rudy,"Biography, Drama, Sport","English, Spanish",22 Oct 1993,7.5
8,111726,80.622281,Scooby-Doo! WrestleMania Mystery,"Animation, Comedy, Family, Mystery",English,25 Mar 2014,6.3
9,89112,80.775261,The Kid from Brooklyn,"Comedy, Musical",English,21 Mar 1946,6.7


In [78]:
rank(bcq_action, distance.minkowski)

,id,score,Title,Genre,Language,Released,imdbRating
0,3340,27.769102,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
1,3638,27.866560,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
2,2270,28.175209,A Century of Cinema,Documentary,English,N/A,5.6
3,113128,28.272356,Return to Life,Documentary,English,20 Jul 1938,6.0
4,77734,28.288343,Confessions of a Gangsta,Action,English,26 Sep 2006,4.3
5,97304,28.448215,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
6,122458,28.585938,Supernova,"Action, Adventure, Drama","English, French",05 Sep 2005,3.7
7,80160,28.630526,The Flintstone Kids' Just Say No Special,Animation,English,15 Sep 1988,5.6
8,5497,28.633619,Our Man Flint,"Action, Adventure, Comedy, Fantasy, Sci-Fi","English, Italian",16 Jan 1966,6.5
9,2683,28.641691,Austin Powers: The Spy Who Shagged Me,"Action, Adventure, Comedy, Crime","English, German",11 Jun 1999,6.6


In [79]:
rank(bcq_action, distance.chebyshev)

,id,score,Title,Genre,Language,Released,imdbRating
0,71254,5.826878,Gamer,"Action, Sci-Fi, Thriller",English,04 Sep 2009,5.8
1,4678,5.872302,UHF,"Comedy, Drama",English,21 Jul 1989,7.0
2,57692,5.921563,That Man from Rio,"Action, Adventure, Comedy, Romance","French, English, Portuguese",05 Feb 1964,7.1
3,45106,5.924949,American Dreamz,"Comedy, Music","English, Mandarin",21 Apr 2006,5.5
4,66152,5.952034,TerrorVision,"Comedy, Horror, Sci-Fi",English,14 Feb 1986,5.7
5,103460,5.971830,Ghoulies Go to College,"Comedy, Fantasy, Horror",English,18 Sep 1991,3.9
6,81132,5.972182,Rubber,"Comedy, Fantasy, Horror",English,10 Nov 2010,5.8
7,4009,5.990061,Talk Radio,Drama,English,13 Jan 1989,7.3
8,6482,5.996605,Dumb and Dumberer: When Harry Met Lloyd,Comedy,English,13 Jun 2003,3.4
9,79599,6.007491,Operation: Endgame,"Action, Comedy",English,19 May 2012,4.9


In [80]:
rank(bcq_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,8878,0.704326,Class of Nuke 'Em High Part II: Subhumanoid Me...,"Comedy, Horror, Sci-Fi",English,12 Apr 1991,4.4
1,3638,0.712082,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
2,3340,0.717532,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
3,5586,0.717617,The Gypsy Moths,"Action, Drama, Romance",English,13 Sep 1969,6.4
4,31950,0.720379,Curse of the Demon,"Fantasy, Horror, Mystery, Thriller",English,01 Jul 1958,7.5
5,66808,0.720789,Far Cry,"Action, Adventure, Sci-Fi",English,02 Oct 2008,3.1
6,110380,0.724224,Nitro Circus: The Movie,"Documentary, Action, Comedy",English,08 Aug 2012,6.7
7,69626,0.724730,Houdini,"Biography, Drama",English,02 Jul 1953,6.9
8,7423,0.725031,Hitman Hart: Wrestling with Shadows,"Documentary, Biography, Sport",English,20 Dec 1998,8.2
9,94122,0.725402,Comic-Con Episode IV: A Fan's Hope,Documentary,English,23 Nov 2012,7.0


In [81]:
rank(bcq_action, distance.cityblock)

,id,score,Title,Genre,Language,Released,imdbRating
0,31950,240.192123,Curse of the Demon,"Fantasy, Horror, Mystery, Thriller",English,01 Jul 1958,7.5
1,1258,249.180908,The Shining,"Drama, Horror",English,13 Jun 1980,8.4
2,110792,250.279404,Ghost Adventures,Documentary,English,N/A,8.0
3,3638,251.641708,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
4,26429,253.122864,Love at First Bite,Comedy,English,27 Apr 1979,6.1
5,3144,253.795853,The Glass Bottom Boat,"Comedy, Romance",English,09 Jun 1966,6.5
6,3340,254.168716,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
7,2793,254.434235,An American Werewolf in Paris,"Comedy, Fantasy, Horror, Thriller","English, French, German",25 Dec 1997,5.1
8,2455,254.552689,The Fly,"Drama, Horror, Sci-Fi",English,15 Aug 1986,7.5
9,104835,254.683899,The Incredible Melting Man,"Horror, Sci-Fi",English,01 Dec 1977,3.9
